In [ ]:
import pandas as pd

# Read Data
df = pd.read_csv("data.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [3]:
import numpy as np

# Preprocess Data
df = df.drop(columns=["id", "Unnamed: 32"])
X_df = df.drop(columns=["diagnosis"])
y_df = df["diagnosis"]

# Convert to numpy
X = X_df.to_numpy()
y = y_df.to_numpy()

In [ ]:
class GMM:
    def __init__(self, k_components, covariance_type='full'):
        self.k_components = k_components
        self.covariance_type = covariance_type

        self.means = None
        self.covariances = None
        self.weights = None
        self.responsibilities = None

        # Attributes for Experiments
        self.log_likelihoods = []
        self.aic = None
        self.bic = None
        self.labels = None

    def fit(self, X, max_iter=100, tol=1e-6):
        n_samples, n_features = X.shape

        # Initialize parameters
        self.initialize_parameters(X)

        log_likelihood_old = None

        for iteration in range(max_iter):
            # E-step
            self.responsibilities = self.e_step(X)

            # M-step
            self.m_step(X)

            # Compute log likelihood
            log_likelihood_new = self.compute_log_likelihood(X)
            self.log_likelihoods.append(log_likelihood_new)

            # Check for convergence
            if log_likelihood_old is not None and abs(log_likelihood_new - log_likelihood_old) < tol:
                break
            log_likelihood_old = log_likelihood_new

        # Compute AIC and BIC
        self.compute_aic_bic(X)

        # Assign labels based on responsibilities
        self.labels = np.argmax(self.responsibilities, axis=1)

    def initialize_parameters(self, X):
        n_samples, n_features = X.shape
        random_indices = np.random.choice(n_samples, self.k_components, replace=False)
        self.means = X[random_indices]

        self.weights = np.ones(self.k_components) / self.k_components
        global_cov = np.cov(X, rowvar=False)

        if self.covariance_type == 'full':
            self.covariances = np.array([global_cov for _ in range(self.k_components)])
        elif self.covariance_type == 'tied':
            self.covariances = global_cov
        elif self.covariance_type == 'diag':
            diag = np.diag(global_cov)
            self.covariances = np.array([diag for _ in range(self.k_components)])
        elif self.covariance_type == 'spherical':
            var = np.mean(np.diag(global_cov))
            self.covariances = np.array([var for _ in range(self.k_components)])
    
    def e_step(self, X):
        n_samples, n_features = X.shape
        log_numerators = np.zeros((n_samples, self.k_components))

        for k in range(self.k_components):
            if self.covariance_type == 'full':
                cov_k = self.covariances[k]
            elif self.covariance_type == 'tied':
                cov_k = self.covariances
            elif self.covariance_type == 'diag':
                cov_k = self.covariances[k]
            elif self.covariance_type == 'spherical':
                cov_k = self.covariances[k]
            
            log_likelihood = self.log_gaussian(X, k, cov_k)
            log_numerators[:, k] = np.log(self.weights[k] + 1e-10) + log_likelihood

        a_max = np.max(log_numerators, axis=1, keepdims=True)
        shifted_numerators = log_numerators - a_max
        numerators = np.exp(shifted_numerators)

        denominator = np.sum (numerators, axis=1, keepdims=True)
        responsibilities = numerators / denominator

        return responsibilities 
        
    def log_gaussian(self, X, k, cov):
        mean = self.means[k]
        _, n_features = X.shape

        if self.covariance_type in ['spherical', 'diag']:
            cov_stable = cov + 1e-6

            if self.covariance_type == 'spherical':
                log_det = n_features * np.log(cov_stable)
                inverse = 1.0 / cov_stable
            else:
                log_det = np.sum(np.log(cov_stable))
                inverse = 1.0 / cov_stable

            diff = X - mean
            quadratic_term = np.sum ((diff**2) * inverse, axis = 1)
        
        else:
            cov_stable = cov + np.eye(n_features) * 1e-6

            _, log_det = np.linalg.slogdet(cov_stable)
            inverse = np.linalg.inv(cov_stable)

            diff = X - mean
            quadratic_term = np.sum(np.dot(diff, inverse)*diff, axis=1)
        
        constant = -0.5 * n_features * np.log(2*np.pi)
        return constant - 0.5 * log_det - 0.5 * quadratic_term
    
    def m_step (self, X):
        n_samples, n_features = X.shape

        # Update Weights (PI)
        sum_res = np.sum(self.responsibilities, axis=0)
        self.weights = sum_res / n_samples

        # Update Means
        self.means = np.dot(self.responsibilities.T, X) / sum_res.reshape(-1, 1)

        # Update Covariances
        if self.covariance_type == 'full':
            self.covariances = np.zeros((self.k_components, n_features, n_features))
            for k in range(self.k_components):
                diff = X - self.means[k]
                weighted_diff = self.responsibilities[:, k].reshape(-1, 1) * diff
                self.covariances[k] = np.dot(weighted_diff.T, diff) / sum_res[k]
                self.covariances[k].flat[::n_features + 1] += 1e-6
        
        elif self.covariance_type == 'tied':
            cov = np.zeros((n_features, n_features))
            for k in range(self.k_components):
                diff = X - self.means[k]
                weighted_diff = self.responsibilities[:, k].reshape(-1, 1) * diff
                cov += np.dot(weighted_diff.T, diff)
            self.covariances = cov / n_samples
            self.covariances.flat[::n_features + 1] += 1e-6
        
        elif self.covariance_type == 'diag':
            self.covariances = np.zeros((self.k_components, n_features))
            for k in range(self.k_components):
                diff = X - self.means[k]
                weighted_diff_sq = self.responsibilities[:, k].reshape(-1, 1) * (diff ** 2)
                self.covariances[k] = np.sum(weighted_diff_sq, axis=0) / sum_res[k] + 1e-6
        
        elif self.covariance_type == 'spherical':
            self.covariances = np.zeros(self.k_components)
            for k in range(self.k_components):
                diff = X - self.means[k]
                weighted_diff_sq = self.responsibilities[:, k].reshape(-1, 1) * (diff ** 2)
                self.covariances[k] = np.sum(weighted_diff_sq) / (n_features * sum_res[k]) + 1e-6

    def compute_log_likelihood(self, X):
        n_samples = X.shape[0]
        log_numerators = np.zeros((n_samples, self.k_components))

        for k in range(self.k_components):
            if self.covariance_type == 'full':
                cov_k = self.covariances[k]
            elif self.covariance_type == 'tied':
                cov_k = self.covariances
            elif self.covariance_type == 'diag':
                cov_k = self.covariances[k]
            elif self.covariance_type == 'spherical':
                cov_k = self.covariances[k]
            
            # Compute log-probability for component k
            log_density = self.log_gaussian(X, k, cov_k)
            log_numerators[:, k] = np.log(self.weights[k] + 1e-10) + log_density

        a_max = np.max(log_numerators, axis=1, keepdims=True)
        return np.sum(a_max + np.log(np.sum(np.exp(log_numerators - a_max), axis=1)))
    
    def compute_aic_bic(self, X):
        n_samples, n_features = X.shape
        log_likelihood = self.log_likelihoods[-1]

        n_params = 0
        # Means parameters (K * D)
        n_params += self.k_components * n_features
        
        # Weights parameters (K - 1)
        n_params += self.k_components - 1

        # Covariance parameters
        if self.covariance_type == 'full':
            # K * (D * (D+1) / 2)
            n_params += self.k_components * n_features * (n_features + 1) // 2
        elif self.covariance_type == 'tied':
            # 1 shared matrix: D * (D+1) / 2
            n_params += n_features * (n_features + 1) // 2
        elif self.covariance_type == 'diag':
            # K * D
            n_params += self.k_components * n_features
        elif self.covariance_type == 'spherical':
            # K
            n_params += self.k_components

        # Compute AIC and BIC
        self.aic = 2 * n_params - 2 * log_likelihood
        self.bic = n_params * np.log(n_samples) - 2 * log_likelihood
